Competition on Kaggle: https://www.kaggle.com/c/how-good-is-your-medium-article/leaderboard

In [1]:
!pip install tqdm

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import os
import codecs
import json
import seaborn as sns
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from datetime import date

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [4]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

In [5]:
def extract_features_and_write(path_to_data,
                               inp_filename, is_train=True):
    ident = 0
    features = ['_id', 'content', 'published', 'title', 'author']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding="utf-8")
                     for feat in features]
 
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:
    
        for line in tqdm_notebook(inp_json_file):
            dict_data = {}
            json_data = read_json_line(line)
            for i in range(len(feature_files)):
                dict_data[ident] = json_data[features[i]]
                feature_files[i].write(str(dict_data) + '\n')
            ident += 1
    for i in feature_files:
        i.close()

In [6]:
PATH_TO_DATA = 'D:\mlcourse\Databases\kaggle_medium'

In [8]:
extract_features_and_write(PATH_TO_DATA, 'train.json', is_train=True)

In [9]:
extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

## Add the following groups of features:

- Tf-Idf with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Tf-Idf with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Time features: publication hour, whether it's morning, day, night, whether it's a weekend
- Bag of authors (i.e. One-Hot-Encoded author names)

#### Extacting data

In [10]:
prefix = 'train'
features = ['_id', 'content', 'published', 'title', 'author']

l_files = [os.path.join(PATH_TO_DATA,
                                       '{}_{}.txt'.format(prefix, feat))   
                     for feat in features]

In [11]:
%%time
df_all_data = pd.DataFrame({})
for k in range(0, len(features)):
    f = codecs.open(l_files[k], "r", "utf-8")
    arr = []
    count = 0
    for line in f:
        count += 1
        arr.append(list(eval(line[:-2]).values())[0])
    f.close()
    ser_buf = pd.Series(arr)
    df_all_data[features[k]] = ser_buf

Wall time: 12min 21s


In [12]:
df_all_data['content'] = df_all_data['content'].map(strip_tags)

Adding a popularity column

In [14]:
data = pd.read_csv('D:\mlcourse\Databases\kaggle_medium/train_log1p_recommends.csv', sep=',')

In [15]:
df_all_data = df_all_data.join(data)

## Strip content: \xa0, \u200b

In [16]:
dddata = df_all_data.copy(deep=True)

In [17]:
df_all_data = dddata.copy(deep=True)

## Division publications by hour, whether it's morning, day, night, whether it's a weekend


In [18]:
df_all_data.head()

,_id,content,published,title,author,id,log_recommends
0,https://medium.com/policy/medium-terms-of-serv...,"Medium Everyone’s stories and ideas Aug 13, 20...",{'$date': '2012-08-13T22:54:53.510Z'},Medium Terms of Service – Medium Policy – Medium,"{'name': None, 'url': 'https://medium.com/@Med...",8,9.01201
1,https://medium.com/policy/amendment-to-medium-...,"Medium Everyone’s stories and ideas Aug 2, 201...",{'$date': '2015-08-03T07:44:50.331Z'},Amendment to Medium Terms of Service Applicabl...,"{'name': None, 'url': 'https://medium.com/@Med...",14,3.49651
2,https://medium.com/@aelcenganda/%E9%96%A9%E6%9...,"Yun-Chen Chien（簡韻真） Nobody in @g0v.tw, PM in s...",{'$date': '2017-02-05T13:08:17.410Z'},走入山與海之間：閩東大刀會和兩岸走私 – Yun-Chen Chien（簡韻真） – Medium,"{'name': None, 'url': 'https://medium.com/@ael...",19,0.69315
3,https://medium.com/what-comes-to-mind/how-fast...,Vaibhav Khulbe Android App Developer | I write...,{'$date': '2017-05-06T08:16:30.776Z'},How fast can a camera get? – What comes to min...,"{'name': None, 'url': 'https://medium.com/@vai...",22,1.38629
4,https://medium.com/what-comes-to-mind/a-game-f...,Vaibhav Khulbe Android App Developer | I write...,{'$date': '2017-06-04T14:46:25.772Z'},A game for the lonely fox – What comes to mind...,"{'name': None, 'url': 'https://medium.com/@vai...",29,1.94591


#### Creating the columns date and time and preprocessing author

In [19]:
def pub_to_time(publicate):
    return publicate[11:-5]

def pub_to_date(publicate):
    return publicate[:10] 

def time_to_sec(time_s):
    return ((int(time_s[:2])* 60 * 60) + (int(time_s[3:5])) * 60 + (int(time_s[6:])))

# may be needed other division
def sec_to_tofd(time_sec):
    x = 3600
    if(time_sec > 18 * x):
        return 'E'
    elif(time_sec > 12 * x):
        return 'D'
    elif(time_sec > 6 * x):
        return 'M'
    else:
        return 'N'
    
def day_of_week(d):
    days = date(int(d[:4]), int(d[5:7]), int(d[8:]))
    return(days.weekday())

def is_weekend(d):
    if (d > 4):
        return True
    return False

def twitter_data(author_data):
    return author_data['twitter']

def url_data(author_data):
    return author_data['url']

def publish_to_str(publ):
    return list(publ.values())[0]

In [20]:
df_all_data['published'] = df_all_data['published'].map(publish_to_str)
df_all_data['date'] = df_all_data['published'].map(pub_to_date)
df_all_data['time'] = df_all_data['published'].map(pub_to_time)
df_all_data['time_sec'] = df_all_data['time'].map(time_to_sec)
df_all_data['time_of_day'] = df_all_data['time_sec'].map(sec_to_tofd)
df_all_data['week_day'] = df_all_data['date'].map(day_of_week)
df_all_data['weekend'] = df_all_data['week_day'].map(is_weekend)
df_all_data['twitter'] = df_all_data['author'].map(twitter_data)
df_all_data['url'] = df_all_data['author'].map(url_data)

In [44]:
# Not here
feats = ['time_of_day', 'weekend', 'log_recommends']
df = df_all_data[feats]
#X_train, y_train = train_test_split(df, random_state=17)

In [45]:
df.head()

,time_of_day,weekend,log_recommends
0,E,False,9.01201
1,M,False,3.49651
2,D,True,0.69315
3,M,True,1.38629
4,D,True,1.94591


#### Creating 'Bag of authors'

In [24]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
df_all_data['url'] = label_encoder.fit_transform(df_all_data['url'])


In [27]:
author_encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform((df_all_data['url'].values).reshape(-1, 1)))
author_encoded_categorical_columns.head()

,0,1,2,3,4,5,6,7,8,9,...,32172,32173,32174,32175,32176,32177,32178,32179,32180,32181
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Creating 'Bag of authors'

In [46]:
for i in feats[:-1]:
    df[i] = label_encoder.fit_transform(df[i])
data_time = pd.DataFrame(onehot_encoder.fit_transform((df[feats[:-1]])))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
data_time.head()

,0,1,2,3,4,5
0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0


,time_of_day,weekend,log_recommends
0,1,0,9.01201
1,2,0,3.49651
2,0,1,0.69315
3,2,1,1.38629
4,0,1,1.94591


## Creating Tf-Idf 

- with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)


In [28]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_title = count_vect.fit_transform(df_all_data.title)

tf_transformer_title = TfidfTransformer(use_idf=False).fit(df_counts_title)
df_tf_title = tf_transformer_title.transform(df_counts_title)

tfidf_transformer = TfidfTransformer()
df_tfidf_title = tfidf_transformer.fit_transform(df_counts_title)

Wall time: 4.31 s


In [40]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_content = count_vect.fit_transform(df_all_data.content)

tf_transformer_content = TfidfTransformer(use_idf=False).fit(df_counts_content)
df_tf_content = tf_transformer_content.transform(df_counts_content)

tfidf_transformer = TfidfTransformer()
df_tfidf_content = tfidf_transformer.fit_transform(df_counts_content)

Wall time: 38min 13s


In [29]:
count_vect_light = CountVectorizer(ngram_range=(1, 2), max_features= 1000)
df_counts_title_light = count_vect_light.fit_transform(df_all_data.title)

tf_transformer_title_light = TfidfTransformer(use_idf=False).fit(df_counts_title_light)
df_tf_title_light = tf_transformer_title_light.transform(df_counts_title_light)

tfidf_transformer = TfidfTransformer()
df_tfidf_title_light = tfidf_transformer.fit_transform(df_counts_title_light)

In [30]:
count_vect_light = CountVectorizer(ngram_range=(1, 2), max_features= 1000)
df_counts_content_light = count_vect_light.fit_transform(df_all_data.content)

tf_transformer_content_light = TfidfTransformer(use_idf=False).fit(df_counts_content_light)
df_tf_content_light = tf_transformer_content_light.transform(df_counts_content_light)

tfidf_transformer = TfidfTransformer()
df_tfidf_content_light = tfidf_transformer.fit_transform(df_counts_content_light)

In [67]:
df_tfidf_content.shape, df_tfidf_title.shape, author_encoded_categorical_columns.shape

((62313, 100000), (62313, 100000), (62313, 32182))

Sparsing train and test (don't divided) data

In [ ]:
%%time
X_train_sparse = hstack([author_encoded_categorical_columns,
                         df_tfidf_content_light, df_tfidf_title_light, data_time]).tocsr()

In [65]:
X_train_sparse.shape, len(df[feats[-1]])

((62313, 34188), 62313)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X_train_sparse, df[feats[-1]], random_state=17)